In [6]:
import numpy as np
import scipy, sys, os, pyproj, glob, re, h5py
import matplotlib.pyplot as plt

from scipy.signal import correlate
from astropy.time import Time

# Import some of the scripts that we have written
import sys
sys.path.append("/home/jovyan/surface_velocity/scripts")
from loading_scripts import atl06_to_dict

%matplotlib widget
%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
datapath = '/home/jovyan/shared/surface_velocity/FIS_ATL06/'

# find all the files in the directory:
# ATL06_files=glob.glob(os.path.join(datapath, 'PIG_ATL06', '*.h5'))

ATL06_files=glob.glob(os.path.join(datapath, '*.h5'))

D_dict={}
error_count=0
for file in ATL06_files:
    try:
        D_dict[file]=atl06_to_dict(file, '/gt2l', index=slice(0, -1, 25), epsg=3031)
    except KeyError as e:
        print(f'file {file} encountered error {e}')
        error_count += 1
print(f"read {len(D_dict)} data files of which {error_count} gave errors")

file /home/jovyan/shared/surface_velocity/FIS_ATL06/processed_ATL06_20190430122344_04920311_003_01.h5 encountered error 'Unable to open object (component not found)'
file /home/jovyan/shared/surface_velocity/FIS_ATL06/processed_ATL06_20181030210407_04920111_003_01.h5 encountered error 'Unable to open object (component not found)'
file /home/jovyan/shared/surface_velocity/FIS_ATL06/processed_ATL06_20190730080323_04920411_003_01.h5 encountered error 'Unable to open object (component not found)'
file /home/jovyan/shared/surface_velocity/FIS_ATL06/processed_ATL06_20190220230230_08320211_003_01.h5 encountered error 'Unable to open object (component not found)'
file /home/jovyan/shared/surface_velocity/FIS_ATL06/processed_ATL06_20190312235510_11380211_003_01.h5 encountered error 'Unable to open object (component not found)'
file /home/jovyan/shared/surface_velocity/FIS_ATL06/processed_ATL06_20181108184743_06280111_003_01.h5 encountered error 'Unable to open object (component not found)'
file

In [11]:
beams = ['gt1l']#,'gt1r','gt2l','gt2r','gt3l','gt3r']

for i,key in enumerate(D_dict.keys()):
    
    rgt=key[78:82]

    D_2l={}
    D_2r={}

    # iterate over the repeat cycles
    for cycle in ['03','04','05','06','07']:
        for filename in glob.glob(os.path.join(datapath, f'*ATL06_*_{rgt}{cycle}*_003*.h5')):
            try:
                # read the left-beam data
                D_2l[filename]=atl06_to_dict(filename,'/gt2l', index=None, epsg=3031)
                # read the right-beam data
                D_2r[filename]=atl06_to_dict(filename,'/gt2r', index=None, epsg=3031)
            except Exception as e:
                print(f'filename={filename}, exception={e}')


    cycles = [] # names of cycles with data
    for filename, Di in D_2l.items():
        cycles += [str(Di['cycle']).zfill(2)]
    cycles.sort()

    if len(cycles)<2:
        continue
    print(i,len(cycles))
    
    ### extract and plot data from all available cycles
    x_atc = {}
    h_li = {}
    h_li_diff = {}
    times = {}
    for cycle in cycles:
        # find Di that matches cycle:
        Di = {}
        x_atc[cycle] = {}
        h_li[cycle] = {}
        h_li_diff[cycle] = {}
        times[cycle] = {}

        filenames = glob.glob(os.path.join(datapath, f'*ATL06_*_{rgt}{cycle}*_003*.h5'))
        for filename in filenames:
            try:
                for beam in beams:
                    Di[filename]=atl06_to_dict(filename,'/'+ beam, index=None, epsg=3031)

                    times[cycle][beam] = Di[filename]['data_start_utc']

                    # extract h_li and x_atc for that section
                    x_atc_tmp = Di[filename]['x_atc']
                    h_li_tmp = Di[filename]['h_li']#[ixs]

                    # segment ids:
                    seg_ids = Di[filename]['segment_id']
    #                 print(len(seg_ids), len(x_atc_tmp))

                    # make a monotonically increasing x vector
                    # assumes dx = 20 exactly, so be carefull referencing back
                    ind = seg_ids - np.nanmin(seg_ids) # indices starting at zero, using the segment_id field, so any skipped segment will be kept in correct location
                    x_full = np.arange(np.max(ind)+1) * 20 + x_atc_tmp[0]
                    h_full = np.zeros(np.max(ind)+1) + np.NaN
                    h_full[ind] = h_li_tmp


                    x_atc[cycle][beam] = x_full
                    h_li[cycle][beam] = h_full


    #                 ### here is where you would put a filter
    #                 # you would want to de-mean and detrend that section first:
    #                 h = h_full
    #                 x = x_full
    #                 h = h - np.nanmean(h) # de-mean
    #                 h = scipy.signal.detrend(h, type = 'linear') # de-trend; need to deal with nans first
    #                 # use scipy.signal.filter to filter

    #                 # differentiate that section of data
                    h_diff = (h_full[1:] - h_full[0:-1]) / (x_full[1:] - x_full[0:-1])
                    h_li_diff[cycle][beam] = h_diff

            except:
                print(f'filename={filename}')
                
                
    n_veloc = len(cycles) - 1
    veloc_number = 0

    segment_length = 3000 # m

    search_width = 500 # m
    dx = 20 # meters between x_atc points

    x1 = 2.915e7#x_atc[cycles[0]][beams[0]][1000] <-- the very first x value in a file; doesn't work, I think b/c nans # 2.93e7
    x1s = x_atc[cycles[veloc_number]][beams[0]][search_width+500:-segment_length-2*search_width:10]
    velocities = {}  
    x_out = np.empty_like(x1s)
    y_out = np.empty_like(x1s)
    for beam in beams:
        velocities[beam] = np.empty_like(x1s)
    for xi,x1 in enumerate(x1s):
        for veloc_number in range(n_veloc):
            cycle1 = cycles[veloc_number]
            cycle2 = cycles[veloc_number+1]
            t1_string = times[cycle1]['gt1l'][0].astype(str) #figure out later if just picking hte first one it ok
            t1 = Time(t1_string)

            t2_string = times[cycle2]['gt1l'][0].astype(str) #figure out later if just picking hte first one it ok
            t2 = Time(t2_string)

            dt = (t2 - t1).jd # difference in julian days

            for beam in beams:
                # cut out small chunk of data at time t1 (first cycle)
                x_full_t1 = x_atc[cycle1][beam]
                ix_x1 = np.arange(len(x_full_t1))[x_full_t1 >= x1][0]
                ix_x2 = ix_x1 + int(np.round(segment_length/dx))      
                x_t1 = x_full_t1[ix_x1:ix_x2]
                h_li1 = h_li_diff[cycle1][beam][ix_x1-1:ix_x2-1] # start 1 index earlier because the data are differentiated

                # cut out a wider chunk of data at time t2 (second cycle)
                x_full_t2 = x_atc[cycle2][beam]
                ix_x3 = ix_x1 - int(np.round(search_width/dx)) # offset on earlier end by # indices in search_width
                ix_x4 = ix_x2 + int(np.round(search_width/dx)) # offset on later end by # indices in search_width
                x_t2 = x_full_t2[ix_x3:ix_x4]
                h_li2 = h_li_diff[cycle2][beam][ix_x3:ix_x4]
                
                try:
                    # correlate old with newer data
                    corr = correlate(h_li1, h_li2, mode = 'valid', method = 'direct') 
                    norm_val = np.sqrt(np.sum(h_li1**2)*np.sum(h_li2**2)) # normalize so values range between 0 and 1
                    corr = corr / norm_val
                except:
                    continue


        #         lagvec = np.arange( -(len(h_li1) - 1), len(h_li2), 1)# for mode = 'full'
        #         lagvec = np.arange( -int(search_width/dx) - 1, int(search_width/dx) +1, 1) # for mode = 'valid'
                lagvec = np.arange(- int(np.round(search_width/dx)), int(search_width/dx) +1,1)# for mode = 'valid'

                shift_vec = lagvec * dx

                if all(np.isnan(corr)):
                    velocities[beam][xi] = np.nan
                else:
                    ix_peak = np.arange(len(corr))[corr == np.nanmax(corr)][0]
                    best_lag = lagvec[ix_peak]
                    best_shift = shift_vec[ix_peak]
                    velocities[beam][xi] = best_shift/(dt/365)
                
                try:
                    x_out[xi] = Di[filename]['x'][(ix_x1+ix_x2)//2]
                    y_out[xi] = Di[filename]['y'][(ix_x1+ix_x2)//2]
                except:
                    x_out[xi] = np.nan
                    y_out[xi] = np.nan
                    
    out = np.array([x_out,y_out,velocities[beam]])
    np.save('/home/jovyan/shared/surface_velocity/Corr_Vel/corr_vel_'+rgt,out)

4 2
filename=/home/jovyan/shared/surface_velocity/FIS_ATL06/processed_ATL06_20190509193251_06340311_003_01.h5
filename=/home/jovyan/shared/surface_velocity/FIS_ATL06/processed_ATL06_20190808151232_06340411_003_01.h5


KeyError: 'gt1l'

In [14]:
x_atc

{'03': {}, '04': {}}